In [1]:
!az login

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code FHMABR5BG to authenticate.
[
  {
    "cloudName": "AzureCloud",
    "id": "63fc0c29-abe6-4170-b3bf-1cf85656b247",
    "isDefault": true,
    "name": "Visual Studio Enterprise",
    "state": "Enabled",
    "tenantId": "4e9843a2-758e-4a3b-8628-2fc4141091af",
    "user": {
      "name": "jmangia@me.com",
      "type": "user"
    }
  }
]


In [31]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
import os

# choose a name for your cluster
compute_name = os.environ.get("AML_COMPUTE_CLUSTER_NAME", "cpucluster")
compute_min_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MIN_NODES", 0)
compute_max_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MAX_NODES", 4)

# This example uses CPU VM. For using GPU VM, set SKU to STANDARD_NC6
vm_size = os.environ.get("AML_COMPUTE_CLUSTER_SKU", "STANDARD_D2_V2")


if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print('found compute target. just use it. ' + compute_name)
else:
    print('creating a new compute target...')
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = vm_size,
                                                                min_nodes = compute_min_nodes, 
                                                                max_nodes = compute_max_nodes)

    # create the cluster
    compute_target = ComputeTarget.create(ws, compute_name, provisioning_config)
    
    # can poll for a minimum number of nodes and for a specific timeout. 
    # if no min node count is provided it will use the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
     # For a more detailed view of current AmlCompute status, use get_status()
    print(compute_target.get_status().serialize())

creating a new compute target...
Creating
Succeeded
AmlCompute wait for completion finished
Minimum number of nodes requested have been provisioned
{'allocationState': 'Steady', 'allocationStateTransitionTime': '2019-04-19T21:18:30.286000+00:00', 'creationTime': '2019-04-19T21:17:12.922263+00:00', 'currentNodeCount': 0, 'errors': None, 'modifiedTime': '2019-04-19T21:18:32.227125+00:00', 'nodeStateCounts': {'idleNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0, 'preparingNodeCount': 0, 'runningNodeCount': 0, 'unusableNodeCount': 0}, 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'targetNodeCount': 0, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


In [19]:
from azureml.core import Workspace, Experiment, Run, Datastore
ws = Workspace.from_config()

Found the config file in: /home/nbuser/library/config.json


In [20]:
experiment_name = "my-first-experiment"
run = list(Experiment(workspace = ws, name = experiment_name).get_runs())[0]

In [21]:
run

Experiment,Id,Type,Status,Details Page,Docs Page
my-first-experiment,75aca6d8-b47a-4c4c-98be-154ddce7b53a,,Completed,Link to Azure Portal,Link to Documentation


In [22]:
#list all datastores registered in current workspace
datastores = ws.datastores
for name, ds in datastores.items():
    print(name, ds.datastore_type)

workspaceblobstore AzureBlob
workspacefilestore AzureFile
challenge1 AzureBlob


In [23]:
# Default datastore (Azure file storage)
#def_data_store = ws.get_default_datastore() 
def_data_store = Datastore(ws, "challenge1")
def_data_store

In [30]:
from azureml.data.data_reference import DataReference
from azureml.pipeline.core import PipelineData

input_data_reference = DataReference(
    datastore=def_data_store,
    data_reference_name="test_data",
    path_on_datastore="microsoft-malware-prediction/test.csv")

output_data = PipelineData(
    "output_data1",
    datastore=def_data_store,
    output_name="output_data1")

In [40]:
%%writefile batch_score.py

def init():

def run(raw_data):


Writing batch_score.py


In [41]:
from azureml.pipeline.steps.python_script_step import PythonScriptStep

firstStep = PythonScriptStep(
    script_name="batch_score.py",
    arguments=["--input", input_data_reference, "--output", output_data],
    inputs=[input_data_reference],
    outputs=[output_data],
    compute_target=compute_target,
    source_directory="."
)

In [43]:
from azureml.pipeline.core import Pipeline

pipeline1 = Pipeline(workspace=ws, steps=[firstStep])
pipeline1.publish(name="Test Pipeline")

Created step batch_score.py [10699675][5ec6a1cb-1d5a-4e23-a4ea-13e03dc59567], (This step will run and generate new outputs)
Created data reference test_data for StepId [875e5fe7][cfe7239c-0f2e-4945-943e-a824ce376816], (Consumers of this data will generate new runs.)


Name,Id,Status,Endpoint
Test Pipeline,241d4613-595a-4836-bd81-66ea9a1d66d7,Active,REST Endpoint


In [46]:
# Submit the pipeline to be run
pipeline_run1 = Experiment(ws, "Test_pipeline_exp").submit(pipeline1)
pipeline_run1.wait_for_completion()


Submitted pipeline run: 64db5dfc-8209-4432-b6d4-503a4a5cf694
RunId: 64db5dfc-8209-4432-b6d4-503a4a5cf694
Link to Portal: https://mlworkspace.azure.ai/portal/subscriptions/63fc0c29-abe6-4170-b3bf-1cf85656b247/resourceGroups/MLHack/providers/Microsoft.MachineLearningServices/workspaces/challenge1/experiments/Test_pipeline_exp/runs/64db5dfc-8209-4432-b6d4-503a4a5cf694
Status: Running
.................................................................................................
Status: Failed
{'runId': '64db5dfc-8209-4432-b6d4-503a4a5cf694', 'status': 'Failed', 'startTimeUtc': '2019-04-19T21:41:13.71873Z', 'endTimeUtc': '2019-04-19T21:50:07.694075Z', 'properties': {'azureml.runsource': 'azureml.PipelineRun', 'runSource': None, 'runType': 'HTTP', 'azureml.parameters': '{}'}, 'logFiles': {'logs/azureml/stderrlogs.txt': 'https://challenge12193430701.blob.core.windows.net/azureml/ExperimentRun/dcid.64db5dfc-8209-4432-b6d4-503a4a5cf694/logs/azureml/stderrlogs.txt?sv=2018-03-28&sr=b&sig=TETjS

'Failed'